In [1]:
# Common default imports
from pprint import pprint
from model_manager import LDAManager
from main.lda.config import LdaGeneratorConfig
from gensim import corpora
import pandas as pd
from gensim.corpora import Dictionary
# Plotting utility for the LDA models
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Stop words identified from the next blocks
stop_words = ['game', 'play', '<game_name>']

# Pre Processing Step
We want to process the data for LDA like:
- Keep NOUNS only as they are the elements that mostly hold aspect value
- Split reviews in sentences (Avoid topic modelling in a too broad sense) (https://aclanthology.org/N10-1122.pdf)

In [ ]:
# LDA does not benefit from repeated words among many documents. So we should clean them as they are stopwords.
corpus_path = "../dataset/output/default_sentences/pre_processed.310k.csv"
ds = pd.read_csv(corpus_path)['comments'].apply(lambda x: x.split(' '))

dictionary = corpora.Dictionary(ds)
doc_frequency = {}

for index in range(len(dictionary)):
    doc_frequency[dictionary.get(index)] = len(ds[[dictionary.get(index) in x for x in ds]])
pd.DataFrame.from_dict(doc_frequency, orient='index')[0].map(lambda x: x / len(ds)).sort_values(ascending=False)

Are 'game'(40%), 'play'(20%) and '\<game_name\>'(11%) stopwords? <br>
For sure 'game' is! What about the other two? I believe they bring no context.
We remove them!

### Default: Sentences slit

In [6]:
# 80k
corpus_path = "../dataset/output/default_sentences/pre_processed.80k.csv"
test_corpus_path = "../dataset/output/default_sentences/pre_processed.80k.test.csv"

model_manager = LDAManager.from_scratch(LdaGeneratorConfig(name="default_sentences_80k"), stop_words)
model = model_manager.get_model(corpus_path, True, True)
res = model_manager.evaluate(test_corpus_path)

pprint(res)

Generating a new compiled model from fs
Model not found. Making a new one.
Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/68390 [00:00<?, ?it/s]

{'coherence': [-17.177862702834112,
               -17.02777543298526,
               -17.290263201442876,
               -16.936958294632383],
 'perplexity': -8.588481933552947,
 'topn': [3, 5, 10, 20]}


In [7]:
dictionary: Dictionary = model.id2word
# Load texts to
texts = pd.read_csv(test_corpus_path)['comments'].apply(lambda x: x.split(' '))

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(model, texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)
# Render the plot
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.190883  0.002691       1        1  10.265110
10     0.143829  0.116062       2        1   9.961937
4     -0.048755  0.034062       3        1   7.992255
2     -0.090078  0.035395       4        1   7.733778
13     0.051645  0.040787       5        1   7.196578
11     0.159590  0.037115       6        1   7.019158
5     -0.059096 -0.064933       7        1   6.711771
7     -0.066562  0.026106       8        1   6.662826
0      0.067464 -0.017909       9        1   6.654468
9      0.039191 -0.040358      10        1   6.594026
8     -0.029328 -0.057865      11        1   6.337182
3     -0.049830 -0.063875      12        1   5.786575
12    -0.034708  0.129836      13        1   5.625053
6      0.107520 -0.177115      14        1   5.459282, topic_info=           Term         Freq        Total Category  logprob  loglift
57         card  2241.000000  2241.000000  Default  30.0000  30.0000
258       great  1187.000000  1187.000000  Default  29.0000  29.0000
79         easy   644.000000   644.000000  Default  28.0000  28.0000
195        dice   521.000000   521.000000  Default  27.0000  27.0000
8        player  2470.000000  2470.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
7          love    42.924259   648.353831  Topic14  -5.2775   0.1929
286  experience    38.811085   352.073171  Topic14  -5.3783   0.7027
17         nice    38.186116   538.036268  Topic14  -5.3945   0.2624
33        board    38.142863   624.151115  Topic14  -5.3956   0.1128
118         fun    39.012879  1550.594174  Topic14  -5.3731  -0.7746

[1161 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
111       1  0.042018  <number>
111       2  0.028012  <number>
111       3  0.028012  <number>
111       4  0.084037  <number>
111       5  0.065362  <number>
...     ...       ...       ...
700      12  0.503348     young
3807      2  0.954814   youtube
5852     14  0.904323        yr
7507      5  0.840289      zany
7420      6  0.878548       zee

[4388 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 11, 5, 3, 14, 12, 6, 8, 1, 10, 9, 4, 13, 7])

Now the bigger dataset for 310k

In [2]:
corpus_path = "../dataset/output/default_sentences/pre_processed.310k.csv"
test_corpus_path = "../dataset/output/default_sentences/pre_processed.310k.test.csv"

model_manager = LDAManager.from_scratch(LdaGeneratorConfig(name="default_sentences"), stop_words)
model = model_manager.get_model(corpus_path, True, True)
res = model_manager.evaluate(test_corpus_path)

pprint(res)

Generating a new compiled model from fs
Model not found. Making a new one.
Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/233897 [00:00<?, ?it/s]

{'coherence': [-14.345296267821746,
               -13.742494545430045,
               -14.779922291032763,
               -15.116752158504827],
 'perplexity': -8.19623024177193,
 'topn': [3, 5, 10, 20]}


In [3]:
dictionary: Dictionary = model.id2word
# Load texts to
texts = pd.read_csv(test_corpus_path)['comments'].apply(lambda x: x.split(' '))

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(model, texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)
# Render the plot
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13    -0.231924 -0.006110       1        1  9.162896
12    -0.100420  0.044382       2        1  8.825932
7      0.203365 -0.140904       3        1  8.540160
2      0.080816  0.064884       4        1  8.042860
1     -0.014630  0.184309       5        1  7.608133
8     -0.158234 -0.158961       6        1  7.512413
9      0.036677  0.174920       7        1  7.412632
10    -0.164390 -0.123739       8        1  7.394272
6      0.072340 -0.155483       9        1  6.960229
11     0.064044  0.096032      10        1  6.736107
0     -0.080141  0.073942      11        1  5.758742
5      0.201921 -0.064437      12        1  5.673783
4      0.051864  0.063850      13        1  5.270739
3      0.038715 -0.052686      14        1  5.101103, topic_info=       Term         Freq        Total Category  logprob  loglift
174    card  7263.000000  7263.000000  Default  30.0000  30.0000
147    good  6269.000000  6269.000000  Default  29.0000  29.0000
142    well  2448.000000  2448.000000  Default  28.0000  28.0000
133    time  4748.000000  4748.000000  Default  27.0000  27.0000
209    love  2371.000000  2371.000000  Default  26.0000  26.0000
..      ...          ...          ...      ...      ...      ...
840   solid   109.440791   623.082266  Topic14  -5.5117   1.2364
128    make   126.310025  1822.839299  Topic14  -5.3684   0.3063
147    good   129.113918  6269.552848  Topic14  -5.3464  -0.9071
24   design   114.543730  1164.904720  Topic14  -5.4661   0.6563
158    need   115.560843  2214.071117  Topic14  -5.4573   0.0229

[1083 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
694       1  0.056123  <number>
694       2  0.017460  <number>
694       4  0.001247  <number>
694       5  0.710891  <number>
694       6  0.004989  <number>
...     ...       ...       ...
207      11  0.018825      year
207      13  0.935478      year
739      10  0.973116         z
8626     10  0.991476       zee
8117     10  0.923992       zen

[4588 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 13, 8, 3, 2, 9, 10, 11, 7, 12, 1, 6, 5, 4])

### NOUN-sentences ds
Filter out any non noun from dataset using the POS tagged processing pipleine

In [ ]:
from pre_processing import extract_pos_ds
import pandas as pd

print("Creating the __noun filtered ds:")
corpus_path = "../dataset/output/pos_tagged_sentence_level/pre_processed.310k.csv"
store_path = "../dataset/output/pos_tagged_sentence_level/pre_processed.310k.noun_only.csv"
extract_pos_ds(pd.read_csv(corpus_path)['comments'], "__noun", store_path)
print("ds created under: " + store_path)

print("Creating the __noun filtered test ds:")
corpus_path = "../dataset/output/pos_tagged_sentence_level/pre_processed.310k.test.csv"
store_path = "../dataset/output/pos_tagged_sentence_level/pre_processed.310k.noun_only.test.csv"
extract_pos_ds(pd.read_csv(corpus_path)['comments'], "__noun", store_path)
print("ds created under: " + store_path)

Now we check if the NOUN only approach works better. <br>
Take nouns only of the POS tagged ds:

In [ ]:
corpus_path = "../dataset/output/pos_tagged_sentence_level/pre_processed.310k.noun_only.csv"
test_corpus_path = "../dataset/output/pos_tagged_sentence_level/pre_processed.310k.noun_only.test.csv"

model_manager = LDAManager.from_scratch(LdaGeneratorConfig(name="default_sentences"), stop_words)
model = model_manager.get_model(corpus_path, False, True)
res = model_manager.evaluate(test_corpus_path)

pprint(res)

In [14]:
dictionary: Dictionary = model.id2word
# Load texts to
texts = pd.read_csv(test_corpus_path)['comments'].apply(lambda x: x.split(' '))

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(model, texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13    -0.233209 -0.005561       1        1  9.198038
12    -0.101986  0.047204       2        1  8.843128
7      0.202171 -0.143089       3        1  8.535858
2      0.080518  0.066422       4        1  8.031112
1     -0.013881  0.184347       5        1  7.622198
8     -0.160835 -0.157964       6        1  7.480469
9      0.038786  0.176612       7        1  7.444389
10    -0.167064 -0.120826       8        1  7.349323
6      0.072377 -0.157634       9        1  6.967530
11     0.065899  0.096497      10        1  6.704844
0     -0.078668  0.072741      11        1  5.783340
5      0.202150 -0.065068      12        1  5.691101
4      0.053988  0.062474      13        1  5.253688
3      0.039752 -0.056154      14        1  5.094982, topic_info=       Term         Freq        Total Category  logprob  loglift
174    card  7263.000000  7263.000000  Default  30.0000  30.0000
147    good  6259.000000  6259.000000  Default  29.0000  29.0000
142    well  2449.000000  2449.000000  Default  28.0000  28.0000
133    time  4740.000000  4740.000000  Default  27.0000  27.0000
209    love  2398.000000  2398.000000  Default  26.0000  26.0000
..      ...          ...          ...      ...      ...      ...
146    come   129.759035  1385.767935  Topic14  -5.3402   0.6086
128    make   133.108429  1822.262821  Topic14  -5.3147   0.3602
24   design   116.141080  1174.316738  Topic14  -5.4511   0.6633
147    good   117.176215  6259.676425  Topic14  -5.4422  -1.0013
158    need   111.652030  2225.565038  Topic14  -5.4905  -0.0155

[1080 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
694       1  0.055129  <number>
694       2  0.017541  <number>
694       4  0.001253  <number>
694       5  0.717934  <number>
694       6  0.005012  <number>
...     ...       ...       ...
207      11  0.019068      year
207      13  0.937261      year
739      10  0.952437         z
8626     10  0.984800       zee
8117     10  0.979445       zen

[4552 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 13, 8, 3, 2, 9, 10, 11, 7, 12, 1, 6, 5, 4])

## NOUN only
On the full review this time so no sentence splitting is performed

In [ ]:
from pre_processing import extract_pos_ds
import pandas as pd

print("Creating the __noun filtered ds:")
corpus_path = "../dataset/output/pos_tagged/pre_processed.310k.csv"
store_path = "../dataset/output/pos_tagged/pre_processed.310k.noun_only.csv"
extract_pos_ds(pd.read_csv(corpus_path)['comments'], "__noun", store_path)
print("ds created under: " + store_path)

print("Creating the __noun filtered test ds:")
corpus_path = "../dataset/output/pos_tagged/pre_processed.310k.test.csv"
store_path = "../dataset/output/pos_tagged/pre_processed.310k.noun_only.test.csv"
extract_pos_ds(pd.read_csv(corpus_path)['comments'], "__noun", store_path)
print("ds created under: " + store_path)

80k test run

In [11]:
# 80k test run
corpus_path = "../dataset/output/pos_tagged/pre_processed.80k.noun_only.csv"
test_corpus_path = "../dataset/output/pos_tagged/pre_processed.80k.noun_only.test.csv"

model_manager = LDAManager.from_scratch(LdaGeneratorConfig(name="noun_only9_80k", topics=9), stop_words)
model = model_manager.get_model(corpus_path, True, True)
res = model_manager.evaluate(test_corpus_path)

pprint(res)

Generating a new compiled model from fs
Model not found. Making a new one.
Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/51995 [00:00<?, ?it/s]

{'coherence': [-12.200767914223142,
               -12.209822157148004,
               -11.71934405929391,
               -12.08168787494773],
 'perplexity': -7.370057944179688,
 'topn': [3, 5, 10, 20]}


In [12]:
dictionary: Dictionary = model.id2word
# Load texts to
texts = pd.read_csv(test_corpus_path)['comments'].apply(lambda x: x.split(' '))

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(model, texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)

vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.070544 -0.037685       1        1  16.009914
5      0.050296 -0.030795       2        1  15.805538
4      0.078301 -0.098102       3        1  13.883903
8     -0.087014  0.017812       4        1  11.683344
3      0.039010 -0.005574       5        1   9.990356
1     -0.165528 -0.101244       6        1   8.970292
2     -0.092859  0.097123       7        1   8.055011
7      0.010182  0.108944       8        1   7.917274
6      0.097068  0.049521       9        1   7.684367, topic_info=          Term         Freq        Total Category  logprob  loglift
32        card  4356.000000  4356.000000  Default  30.0000  30.0000
3          fun  2057.000000  2057.000000  Default  29.0000  29.0000
239  expansion   954.000000   954.000000  Default  28.0000  28.0000
8       player  5058.000000  5058.000000  Default  27.0000  27.0000
59        time  2937.000000  2937.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
5        board    85.251235  1293.994249   Topic9  -4.8971  -0.1539
137        bit    88.506317  1673.652917   Topic9  -4.8596  -0.3737
94         lot    90.425875  2170.545486   Topic9  -4.8382  -0.6122
59        time    74.197348  2937.820047   Topic9  -5.0360  -1.1127
80    building    62.654034   787.296678   Topic9  -5.2051   0.0350

[750 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
184       1  0.226971  ability
184       2  0.094110  ability
184       3  0.268490  ability
184       4  0.033215  ability
184       5  0.066431  ability
...     ...       ...      ...
2046      2  0.084228   zombie
2046      4  0.105285   zombie
2046      6  0.021057   zombie
2046      7  0.652768   zombie
2046      9  0.147399   zombie

[2450 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 6, 5, 9, 4, 2, 3, 8, 7])

Now that the dataset was built and stored train an LDA on it:

In [4]:
corpus_path = "../dataset/output/pos_tagged/pre_processed.310k.noun_only.csv"
test_corpus_path = "../dataset/output/pos_tagged/pre_processed.310k.noun_only.test.csv"

model_manager = LDAManager.from_scratch(LdaGeneratorConfig(name="default_sentences"), stop_words)
model = model_manager.get_model(corpus_path, False, True)
res = model_manager.evaluate(test_corpus_path)

pprint(res)

Generating a new compiled model from scratch
Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/195789 [00:00<?, ?it/s]

{'coherence': [-8.206318645320582,
               -8.092144166145504,
               -8.572682666308848,
               -9.292421386724754],
 'perplexity': -7.193918552296681,
 'topn': [3, 5, 10, 20]}


In [5]:
dictionary: Dictionary = model.id2word
# Load texts to
texts = pd.read_csv(test_corpus_path)['comments'].apply(lambda x: x.split(' '))

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(model, texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)

vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.186977 -0.027139       1        1  14.165697
6      0.151868 -0.040687       2        1   9.435898
1      0.104165  0.129464       3        1   8.514341
2     -0.047553  0.033814       4        1   8.384107
5     -0.155608 -0.039763       5        1   7.256713
7      0.017078  0.018696       6        1   7.113230
3      0.000196  0.078796       7        1   6.845400
9     -0.112893 -0.054376       8        1   6.400776
10    -0.085526  0.059666       9        1   6.359796
4     -0.084226  0.107590      10        1   5.981675
12     0.077980  0.076606      11        1   5.432623
8     -0.096928  0.018351      12        1   4.824361
11     0.150969 -0.162208      13        1   4.671468
13    -0.106498 -0.198811      14        1   4.613913, topic_info=          Term          Freq         Total Category  logprob  loglift
9       player  20121.000000  20121.000000  Default  30.0000  30.0000
87        card  17229.000000  17229.000000  Default  29.0000  29.0000
156       dice   4384.000000   4384.000000  Default  28.0000  28.0000
83   expansion   3693.000000   3693.000000  Default  27.0000  27.0000
382     action   4687.000000   4687.000000  Default  26.0000  26.0000
..         ...           ...           ...      ...      ...      ...
56       board    159.242220   5019.828217  Topic14  -5.1087  -0.3746
386       feel    128.018954    903.564631  Topic14  -5.3270   1.1219
9       player    167.059296  20121.594383  Topic14  -5.0608  -1.7151
83   expansion    145.204889   3693.798325  Topic14  -5.2010  -0.1602
111        way    141.145856   5081.450331  Topic14  -5.2294  -0.5075

[1133 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
3945      14  0.927588        aa
293        1  0.200203   ability
293        2  0.061386   ability
293        3  0.263682   ability
293        4  0.018137   ability
...      ...       ...       ...
10655      2  0.925629  ziggurat
1723       3  0.931097    zombie
1723       4  0.005060    zombie
1723       8  0.005060    zombie
1723      12  0.055663    zombie

[4946 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 7, 2, 3, 6, 8, 4, 10, 11, 5, 13, 9, 12, 14])

Following the experiments I decided to follow the path of nouns only composition document. <br>
The hyperparameters tuning is now the next step.

Noun only on sentences model performs way worse, let's see if going for non LocalLDA but focusing on NOUNS only is viable:

# Hyperparameter tuning results applied
We found the best $K$ for our LDA models applied on the datasets. <br>
As stated in the tuning notebook the NOUN on sentences pipeline produced low coherence models reason for that approach was dropped.

### Default sentences
Best found $K \in \{7, 9\}$ <br>
Let's try training two models on the full data and compare them.

In [21]:
corpus_path = "../dataset/output/default_sentences/pre_processed.310k.csv"
test_corpus_path = "../dataset/output/default_sentences/pre_processed.310k.test.csv"

runs = [
    dict(config=LdaGeneratorConfig(name="sentences-K7", topics=7), K=7, results=[], model=None),
    dict(config=LdaGeneratorConfig(name="sentences-K9", topics=9), K=9, results=[], model=None)
]

In [22]:
import numpy as np

corpus = pd.read_csv("../dataset/output/default_sentences/pre_processed.310k.csv")
folds = np.array_split(corpus, 5)
train = pd.concat([folds[index] for index in range(len(folds)) if index != 2])
train

,comments
0,like art work edition
1,provisional rating base play
2,<game_name> game try like success
3,theme present certainly engage
4,intense strategic game goa aos <game_name> pri...
...,...
233892,pretty clear spirit create equal potentially s...
233893,say doubt example point inequality look make c...
233894,kind randomness okay like <game_name> fine mer...
233895,hit table stay game


In [24]:
for run in runs:
    model_manager = LDAManager.from_scratch(run['config'], stop_words)
    print(model_manager.config)
    run['model'] = model_manager.get_model(train)
    run['results'] = model_manager.evaluate(test_corpus_path)
    # Print the results just cause
    pprint(run['results'])

LdaGeneratorConfig(name='sentences-K7', topics=7, random_state=42, passes=10, alpha='symmetric', eta=None, output_folder='./output')
LdaGeneratorConfig(name='fs', topics=7, random_state=42, passes=10, alpha='symmetric', eta=None, output_folder='./output')
LdaGeneratorConfig(name='sentences-K7', topics=7, random_state=42, passes=10, alpha='symmetric', eta=None, output_folder='./output')
Generating a new compiled model from scratch
Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/187118 [00:00<?, ?it/s]

KeyboardInterrupt: 

Compare the two visualizations

In [19]:
pyLDAvis.enable_notebook()

In [20]:
dictionary: Dictionary = runs[0]['model'].id2word
# Load texts to
texts = pd.read_csv(test_corpus_path)['comments'].apply(lambda x: x.split(' '))

# Visualize the topics
vis = gensimvis.prepare(runs[0]['model'], texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.056613 -0.113578       1        1  18.763109
5      0.216847 -0.027034       2        1  17.106155
6      0.004569  0.169608       3        1  15.257160
2      0.097028 -0.063676       4        1  13.953777
4      0.010685  0.029412       5        1  12.741563
0     -0.156682 -0.125624       6        1  11.496652
3     -0.115834  0.130891       7        1  10.681584, topic_info=          Term         Freq        Total Category  logprob  loglift
174       card  7349.000000  7349.000000  Default  30.0000  30.0000
56        feel  4119.000000  4119.000000  Default  29.0000  29.0000
2         like  7821.000000  7821.000000  Default  28.0000  28.0000
61       great  3900.000000  3900.000000  Default  27.0000  27.0000
152  expansion  1653.000000  1653.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
128       make   240.276660  1818.980200   Topic7  -5.4644   0.2124
114     player   291.100549  8526.826429   Topic7  -5.2725  -1.1407
174       card   254.219550  7349.384463   Topic7  -5.4080  -1.1275
147       good   228.618083  6232.477673   Topic7  -5.5141  -1.0688
56        feel   202.294825  4119.405580   Topic7  -5.6364  -0.7771

[597 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
694       1  0.806778  <number>
694       2  0.001255  <number>
694       3  0.025094  <number>
694       4  0.001255  <number>
694       5  0.097867  <number>
...     ...       ...       ...
207       1  0.152183      year
207       2  0.001449      year
207       4  0.002899      year
207       5  0.681201      year
207       6  0.160879      year

[1842 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 6, 7, 3, 5, 1, 4])

In [21]:
dictionary: Dictionary = runs[1]['model'].id2word
# Load texts to
texts = pd.read_csv(test_corpus_path)['comments'].apply(lambda x: x.split(' '))

# Visualize the topics
vis = gensimvis.prepare(runs[1]['model'], texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.134433 -0.061884       1        1  18.903181
1      0.154963  0.096827       2        1  14.680141
2      0.036757  0.143618       3        1  11.109808
7     -0.211346 -0.001793       4        1  10.750159
6     -0.113136 -0.008747       5        1   9.770296
0      0.136923 -0.096546       6        1   9.325863
5     -0.153015  0.075909       7        1   9.046540
4      0.049697  0.074174       8        1   8.800133
3     -0.035275 -0.221559       9        1   7.613879, topic_info=      Term         Freq        Total Category  logprob  loglift
174   card  7387.000000  7387.000000  Default  30.0000  30.0000
2     like  7863.000000  7863.000000  Default  29.0000  29.0000
142   well  2424.000000  2424.000000  Default  28.0000  28.0000
70     fun  5264.000000  5264.000000  Default  27.0000  27.0000
147   good  6286.000000  6286.000000  Default  26.0000  26.0000
..     ...          ...          ...      ...      ...      ...
297  token   161.135434   469.111875   Topic9  -5.5254   1.5066
128   make   210.830718  1821.431210   Topic9  -5.2566   0.4189
147   good   249.334413  6286.801823   Topic9  -5.0888  -0.6522
158   need   199.309058  2196.776588   Topic9  -5.3128   0.1753
292    set   165.581901  1784.498762   Topic9  -5.4982   0.1978

[736 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
694       1  0.044023  <number>
694       2  0.665378  <number>
694       3  0.003773  <number>
694       4  0.001258  <number>
694       5  0.023898  <number>
...     ...       ...       ...
1430      6  0.006852     young
1430      7  0.815443     young
1430      8  0.178164     young
8117      6  1.001813       zen
1974      6  0.976408       zoo

[2560 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 2, 3, 8, 7, 1, 6, 5, 4])

Map to gold standards and/or recognize the associated aspect

### NOUN only
The best found $K=?$ <br>
Let's try training two models on the full data and compare them.


In [4]:
corpus_path = "../dataset/output/pos_tagged/pre_processed.310k.noun_only.csv"
test_corpus_path = "../dataset/output/pos_tagged/pre_processed.310k.noun_only.test.csv"

runs = [
    dict(config=LdaGeneratorConfig(name="noun-K7", topics=7), K=7, results=[], model=None),
    dict(config=LdaGeneratorConfig(name="noun-K9", topics=10), K=10, results=[], model=None),
]

In [6]:
for run in runs:
    model_manager = LDAManager.from_scratch(run['config'], stop_words)
    run['model'] = model_manager.get_model(corpus_path)
    run['results'] = model_manager.evaluate(test_corpus_path)
    # Print the results just cause
    pprint(run['results'])

Generating a new compiled model from scratch
Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/195789 [00:00<?, ?it/s]

{'coherence': [-7.948833081292833,
               -6.802857928452987,
               -6.95858608877034,
               -7.2892432162160405],
 'perplexity': -6.99375285229275,
 'topn': [3, 5, 10, 20]}
Generating a new compiled model from scratch
Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/195789 [00:00<?, ?it/s]

{'coherence': [-8.906816819996525,
               -7.403514085571197,
               -7.1920301854734685,
               -8.324668992793418],
 'perplexity': -7.056838863124454,
 'topn': [3, 5, 10, 20]}


Visualize the results:

In [10]:
pyLDAvis.enable_notebook()

In [12]:
dictionary: Dictionary = runs[0]['model'].id2word
# Load texts to
texts = pd.read_csv(test_corpus_path)['comments'].apply(lambda x: x.split(' '))

# Visualize the topics
vis = gensimvis.prepare(runs[0]['model'], texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.176886  0.039291       1        1  20.984874
6     -0.108445  0.072347       2        1  15.469234
2      0.041606 -0.002107       3        1  14.063203
3      0.032273  0.023256       4        1  13.629155
1     -0.090313 -0.058556       5        1  12.550242
5      0.213602  0.122682       6        1  11.785681
4      0.088163 -0.196913       7        1  11.517612, topic_info=           Term          Freq         Total Category  logprob  loglift
9        player  20213.000000  20213.000000  Default  30.0000  30.0000
87         card  16961.000000  16961.000000  Default  29.0000  29.0000
83    expansion   3665.000000   3665.000000  Default  28.0000  28.0000
382      action   4660.000000   4660.000000  Default  27.0000  27.0000
28         rule   6977.000000   6977.000000  Default  26.0000  26.0000
..          ...           ...           ...      ...      ...      ...
56        board    626.894157   5069.600425   Topic7  -4.6532   0.0711
26     mechanic    620.300968   5124.399663   Topic7  -4.6637   0.0497
34   collection    391.951266   1777.030287   Topic7  -5.1228   0.6497
17        thing    477.321902   3977.040211   Topic7  -4.9258   0.0412
223   character    359.685925   2051.302563   Topic7  -5.2087   0.4203

[588 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
293       1  0.272122  ability
293       2  0.083460  ability
293       3  0.035769  ability
293       4  0.133256  ability
293       5  0.413793  ability
...     ...       ...      ...
4896      7  0.982500       yr
3902      4  0.957089      zen
1723      3  0.015286   zombie
1723      5  0.978296   zombie
1723      7  0.005095   zombie

[1765 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 7, 3, 4, 2, 6, 5])

In [13]:
dictionary: Dictionary = runs[1]['model'].id2word
# Load texts to
texts = pd.read_csv(test_corpus_path)['comments'].apply(lambda x: x.split(' '))

# Visualize the topics
vis = gensimvis.prepare(runs[1]['model'], texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.184904 -0.004090       1        1  17.766073
6      0.143070  0.043100       2        1  12.506911
7      0.006943  0.099559       3        1  11.456238
1      0.128875 -0.075996       4        1  11.447642
2     -0.028692  0.046133       5        1  10.860257
3      0.005914  0.032234       6        1   9.711551
5     -0.199147  0.153055       7        1   9.489973
8     -0.134669 -0.137988       8        1   8.904077
4     -0.107197 -0.156007       9        1   7.857277, topic_info=           Term          Freq         Total Category  logprob  loglift
87         card  17075.000000  17075.000000  Default  30.0000  30.0000
9        player  20187.000000  20187.000000  Default  29.0000  29.0000
83    expansion   3671.000000   3671.000000  Default  28.0000  28.0000
156        dice   4459.000000   4459.000000  Default  27.0000  27.0000
97          fun   8144.000000   8144.000000  Default  26.0000  26.0000
..          ...           ...           ...      ...      ...      ...
56        board    367.572063   5061.704613   Topic9  -4.8046  -0.0788
178    gameplay    313.607095   2699.529972   Topic9  -4.9634   0.3910
1       edition    244.141995   1435.552849   Topic9  -5.2138   0.7722
4        choice    253.519578   2200.455600   Topic9  -5.1761   0.3828
34   collection    228.462195   1781.991011   Topic9  -5.2801   0.4896

[741 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
293       1  0.213821  ability
293       2  0.074069  ability
293       3  0.067780  ability
293       4  0.395498  ability
293       5  0.030745  ability
...     ...       ...      ...
1723      4  0.851713   zombie
1723      5  0.015392   zombie
1723      7  0.035916   zombie
1723      8  0.092354   zombie
2675      2  0.995767      zoo

[2570 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 7, 8, 2, 3, 4, 6, 9, 5])

Map to gold standards and/or recognize the associated aspect